### Este notebook contém os comandos para estruturar o esquema silver.
1. Criar e popular a tabela *regiao*.

* A tabela *bpc.silver.regiao* é criada e populada a partir da tabela *bpc.bronze.municipios_ibge*.

In [0]:
%sql

-- 1. Cria a tabela 'regiao' no esquema 'silver' do catálogo 'bpc',
--    realiza a conversão de tipos, renomeia colunas, e popula os dados da tabela bronze.
CREATE OR REPLACE TABLE bpc.silver.regiao
USING delta
COMMENT 'Tabela de regiões do Brasil.' 
AS
SELECT DISTINCT
    CAST(CD_REGIAO AS INT) AS CD_REGIAO,
    NM_REGIAO AS NM_REGIAO,
    SIGLA_RG AS SIGLA_RG
FROM
    bpc.bronze.municipios_ibge;

-- 2. Adiciona comentários nas colunas após a criação da tabela.
-- Nota: O CREATE TABLE AS SELECT não permite definir comentários de coluna diretamente na instrução.
ALTER TABLE bpc.silver.regiao CHANGE COLUMN CD_REGIAO CD_REGIAO INT
    COMMENT 'Código da região do Brasil.';

ALTER TABLE bpc.silver.regiao CHANGE COLUMN NM_REGIAO NM_REGIAO STRING
    COMMENT 'Nome da região do Brasil.';

ALTER TABLE bpc.silver.regiao CHANGE COLUMN SIGLA_RG SIGLA_RG STRING
    COMMENT 'Sigla da região do Brasil.';

-- 3. Cria a chave primária (CD_REGIAO)
ALTER TABLE bpc.silver.regiao
ALTER COLUMN CD_REGIAO SET NOT NULL;

ALTER TABLE bpc.silver.regiao
ADD CONSTRAINT regiao_pk PRIMARY KEY (CD_REGIAO);

-- 4. Visualiza as 3 primeiras linhas da tabela regiao recém criada.
SELECT * 
FROM bpc.silver.regiao;

CD_REGIAO,NM_REGIAO,SIGLA_RG
5,Centro-oeste,CO
1,Norte,N
4,Sul,S


2. Tabela *regiao* criada.

![regiao](imagens/04-Esquema_silver-Tabela_regiao.png)

3. Criar e popular a tabela *unidade_federacao*.

* A tabela *bpc.silver.unidade_federacao* é criada e populada a partir da tabela *bpc.bronze.municipios_ibge*.

In [0]:
%sql

-- 1. Cria a tabela 'unidade_federacao' no esquema 'silver' do catálogo 'bpc',
--    realiza a conversão de tipos, renomeia colunas, e popula os dados da tabela bronze.
CREATE OR REPLACE TABLE bpc.silver.unidade_federacao
USING delta
COMMENT 'Tabela de unidades federativas do Brasil.' 
AS
SELECT DISTINCT
    CAST(CD_UF AS INT) AS CD_UF,
    NM_UF AS NM_UF,
    SIGLA_UF AS SIGLA_UF,
    CAST(CD_REGIAO AS INT) AS CD_REGIAO
FROM
    bpc.bronze.municipios_ibge;

-- 2. Adiciona comentários nas colunas após a criação da tabela.
-- Nota: O CREATE TABLE AS SELECT não permite definir comentários de coluna diretamente na instrução.
ALTER TABLE bpc.silver.unidade_federacao CHANGE COLUMN CD_UF CD_UF INT
    COMMENT 'Código da unidade federativa do Brasil.';

ALTER TABLE bpc.silver.unidade_federacao CHANGE COLUMN NM_UF NM_UF STRING
    COMMENT 'Nome da unidade federativa do Brasil.';

ALTER TABLE bpc.silver.unidade_federacao CHANGE COLUMN SIGLA_UF SIGLA_UF STRING
    COMMENT 'Sigla da unidade federativa do Brasil.';

ALTER TABLE bpc.silver.unidade_federacao CHANGE COLUMN CD_REGIAO CD_REGIAO INT
    COMMENT 'Código da região (FK da tabela regiao)';

-- 3. Cria a chave primária (CD_UF)
ALTER TABLE bpc.silver.unidade_federacao
ALTER COLUMN CD_UF SET NOT NULL;

ALTER TABLE bpc.silver.unidade_federacao
ADD CONSTRAINT unidade_federacao_pk PRIMARY KEY (CD_UF);

-- 4. Cria a chave estrangeira (CD_REGIAO)
ALTER TABLE bpc.silver.unidade_federacao
ALTER COLUMN CD_REGIAO SET NOT NULL;

ALTER TABLE bpc.silver.unidade_federacao
ADD CONSTRAINT unidade_federacao_fk_regiao 
    FOREIGN KEY (CD_REGIAO) 
    REFERENCES bpc.silver.regiao (CD_REGIAO);

-- 5. Visualiza as 3 primeiras linhas da tabela unidade_federacao recém criada.
SELECT *
FROM bpc.silver.unidade_federacao
LIMIT 3;

CD_UF,NM_UF,SIGLA_UF,CD_REGIAO
43,Rio Grande do Sul,RS,4
41,Paraná,PR,4
11,Rondônia,RO,1


4. Tabela *unidade_federacao* criada.

![Unidade Federacao](imagens/04-Esquema_silver-Tabela_unidade_federacao.png)

5. Criar e popular a tabela *municipios_ibge*.

* A tabela *bpc.silver.municipios_ibge* é criada e populada a partir da tabela *bpc.bronze.municipios_ibge*.

In [0]:
%sql

-- 1. Cria a tabela 'municipipios_ibge' no esquema 'silver' do catálogo 'bpc',
--    realiza a conversão de tipos, renomeia colunas, e popula os dados da tabela bronze.
CREATE OR REPLACE TABLE bpc.silver.municipios_ibge
USING delta
COMMENT 'Tabela de municípios do IBGE.' 
AS
SELECT
    CAST(SUBSTRING(CAST(CD_MUN AS STRING), 1, 6) AS INT) AS CD_MUN,
    NM_MUN AS NM_MUN,
    CAST(CD_UF AS INT) AS CD_UF,
    CASE 
        WHEN TOCA_LIM = 'SIM' THEN 1
        WHEN TOCA_LIM = 'NÃO' THEN 0
        ELSE NULL
    END AS TOCA_LIM,
    CASE 
        WHEN FAIXA_SEDE = 'sim' THEN 1
        WHEN FAIXA_SEDE = 'não' THEN 0
        ELSE NULL
    END AS FAIXA_SEDE,
    CID_GEMEA AS CID_GEMA
FROM bpc.bronze.municipios_ibge;

-- !!! ATENÇÃO !!! ---
-- Exclue valor inconsistente na tabela.
DELETE FROM bpc.silver.municipios_ibge
WHERE CD_MUN = 430000

-- 2. Adiciona comentários nas colunas após a criação da tabela.
-- Nota: O CREATE TABLE AS SELECT não permite definir comentários de coluna diretamente na instrução.
ALTER TABLE bpc.silver.municipios_ibge CHANGE COLUMN CD_MUN CD_MUN INT
    COMMENT 'Código do município (IBGE).';

ALTER TABLE bpc.silver.municipios_ibge CHANGE COLUMN NM_MUN NM_MUN STRING
    COMMENT 'Nome do município.';

ALTER TABLE bpc.silver.municipios_ibge CHANGE COLUMN CD_UF CD_UF INT
    COMMENT 'Código da unidade federativa do Brasil.';

ALTER TABLE bpc.silver.municipios_ibge CHANGE COLUMN TOCA_LIM TOCA_LIM INT
    COMMENT 'Indica se o limite municipal toca o limite internacional.';

ALTER TABLE bpc.silver.municipios_ibge CHANGE COLUMN FAIXA_SEDE FAIXA_SEDE INT
    COMMENT 'Indica se a sede municipal se encontra dentro da Faixa de Fronteira.';

ALTER TABLE bpc.silver.municipios_ibge CHANGE COLUMN CID_GEMA CID_GEMA STRING
    COMMENT 'Indica se a sede municipal é uma Cidade Gêmea. Caso preenchido, sim';

-- 3. Cria a chave primária (CD_MUN)
ALTER TABLE bpc.silver.municipios_ibge
ALTER COLUMN CD_MUN SET NOT NULL;

ALTER TABLE bpc.silver.municipios_ibge
ADD CONSTRAINT municipios_ibge_pk PRIMARY KEY (CD_MUN);

-- 4. Cria a chave estrangeira (CD_UF)
ALTER TABLE bpc.silver.municipios_ibge
ALTER COLUMN CD_UF SET NOT NULL;

ALTER TABLE bpc.silver.municipios_ibge
ADD CONSTRAINT municipios_ibge_fk_unidade_federacao 
    FOREIGN KEY (CD_UF) 
    REFERENCES bpc.silver.unidade_federacao(CD_UF);

-- 5. Visualiza as 3 primeiras linhas da tabela municipios_ibge recém criada.
SELECT *
FROM bpc.silver.municipios_ibge
LIMIT 10;

CD_MUN,NM_MUN,CD_UF,TOCA_LIM,FAIXA_SEDE,CID_GEMA
110001,Alta Floresta D'Oeste,11,1,1,null
110003,Cabixi,11,1,1,null
110005,Cerejeiras,11,0,1,null
110006,Colorado do Oeste,11,0,1,null
110007,Corumbiara,11,0,1,null
110008,Costa Marques,11,1,1,null
110010,Guajará-Mirim,11,1,1,Portaria N 2.507 de 05/10/2021 do Ministrio do D
110014,Nova Brasilândia D'Oeste,11,0,1,null
110018,Pimenta Bueno,11,0,0,null
110020,Porto Velho,11,1,0,null


6. Tabela *municipios_ibge* criada.

![Municipios ibge](imagens/04-Esquema_silver-Tabela_municipios_ibge.png)

7. Criar e popular a tabela *municipios_pagador*.

* A tabela *bpc.silver.municipios_pagador* é criada e populada a partir da tabela *bpc.bronze.beneficiarios_bpc*.

In [0]:
%sql

-- 1. Cria a tabela 'municipios_pagador' no esquema 'silver' do catálogo 'bpc',
--    realiza a conversão de tipos, renomeia colunas, e popula os dados da tabela bronze.
--    Seleciona-se só os municípios de interesse.
CREATE OR REPLACE TABLE bpc.silver.municipios_pagador
USING delta
COMMENT 'Tabela de municípios do BPC, contendo dados por município, mês, tipo de benefício e valores.' 
AS
SELECT
    CAST(bpc_bronze.ibge AS INT) AS Cod_Mun, 
    TO_DATE(bpc_bronze.anomes, 'yyyyMM') AS Data_Pag, -- Converte STRING '202505' para DATE e depois Databricks pode gerenciar como DATETIME se necessário, mas DATE é mais comum para ano/mês.
    CAST(bpc_bronze.bpc_ben AS INT) AS bpc_ben,
    CAST(bpc_bronze.bpc_pcd_ben AS INT) AS bpc_pcd_ben,
    CAST(bpc_bronze.bpc_idoso_ben AS INT) AS bpc_idoso_ben,
    CAST(bpc_bronze.bpc_pcd_val AS DECIMAL(18,2)) AS bpc_pcd_val,
    CAST(bpc_bronze.bpc_idoso_val AS DECIMAL(18,2)) AS bpc_idoso_val,
    CAST(bpc_bronze.bpc_val AS DECIMAL(18,2)) AS bpc_val
FROM
    bpc.bronze.beneficiarios_bpc bpc_bronze
INNER JOIN
    bpc.silver.municipios_ibge ibge
ON
    CAST(bpc_bronze.ibge AS INT) = ibge.CD_MUN; -- Seleciona-se só os municípios de interesse.

-- 2. Adiciona comentários nas colunas após a criação da tabela.
-- Nota: O CREATE TABLE AS SELECT não permite definir comentários de coluna diretamente na instrução.

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN Cod_Mun Cod_Mun INT
    COMMENT 'Código IBGE do município ao qual o dado se refere.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN Data_Pag Data_Pag DATE
    COMMENT 'Ano e mês de referência do dado.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN bpc_ben bpc_ben INT
    COMMENT 'Quantidade **total** de beneficiários do BPC.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN bpc_pcd_ben bpc_pcd_ben INT
    COMMENT 'Quantidade de beneficiários do BPC na categoria **Pessoa com Deficiência (PcD)**.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN bpc_idoso_ben bpc_idoso_ben INT
    COMMENT 'Quantidade de beneficiários do BPC na categoria **Idoso**.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN bpc_pcd_val bpc_pcd_val DECIMAL(18,2)
    COMMENT '**Valor total** pago para beneficiários PcD do BPC no período.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN bpc_idoso_val bpc_idoso_val DECIMAL(18,2)
    COMMENT '**Valor total** pago para beneficiários Idosos do BPC no período.';

ALTER TABLE bpc.silver.municipios_pagador CHANGE COLUMN bpc_val bpc_val DECIMAL(18,2)
    COMMENT '**Valor total** pago de BPC para todas as categorias.';

-- 3. Cria a chave primária composta (Cod_Mun, Data_Pag)
ALTER TABLE bpc.silver.municipios_pagador
ALTER COLUMN Cod_Mun SET NOT NULL;

ALTER TABLE bpc.silver.municipios_pagador
ALTER COLUMN Data_Pag SET NOT NULL;

ALTER TABLE bpc.silver.municipios_pagador
ADD CONSTRAINT municipios_pagador_pk PRIMARY KEY (Cod_Mun, Data_Pag);

-- 4. Cria a chave estrangeira (Cod_Mun)
ALTER TABLE bpc.silver.municipios_pagador
ADD CONSTRAINT municipios_pagador_fk_municipios_ibge
    FOREIGN KEY (Cod_Mun) 
    REFERENCES bpc.silver.municipios_ibge(CD_MUN);

-- 5. Visualiza as 3 primeiras linhas da tabela municipios_pagador recém-criada
SELECT *
FROM bpc.silver.municipios_pagador
LIMIT 3;

Cod_Mun,Data_Pag,bpc_ben,bpc_pcd_ben,bpc_idoso_ben,bpc_pcd_val,bpc_idoso_val,bpc_val
110001,2025-10-01,690,581,109,881976.58,165462.00,1047438.58
110003,2025-10-01,105,86,19,130553.28,28842.00,159395.28
110005,2025-10-01,573,451,122,684633.21,185196.00,869829.21


8. Tabela *municipios_pagador* criada.

![Municipios pagador](imagens/04-Esquema_silver-Tabela_municipios_pagador_v2.png)

9. Diagrama de Entidade-Relacionamento do Esquema **silver**.

![DER](imagens/04-Esquema_silver-DER.png)

10. Esquema **silver** criado.